## Word Vectorisation Notebook
Below is the code used for generating various embeddings used in the neural models.

### Imported libraries

In [1]:
import os
import urllib.request
import numpy as np
import zipfile
import tensorflow as tf
import collections
import random
import h5py as h5py


from keras.models import Sequential,Model
from keras.optimizers import RMSprop
from keras.layers import Embedding,LSTM,Dense,Lambda,merge,Input
from keras.callbacks import TensorBoard,ModelCheckpoint,Callback
from keras import backend as K

/Applications/anaconda/envs/nlp/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


### Functions defined
```python
maybe_download(filename)
#Downloads a file if not present.

read_data(filename)
#Extract the first file enclosed in a zip file as a list of words.

build_dataset(words, n_words)
#Process raw inputs into a dataset.

generate_batches(data, size, contextWidth, negativeSize)
#Returns batches of input words with their contexts and a set of negative samples.
```


In [ ]:
def maybe_download(url, filename):
  """Download a file if not present."""
  if not os.path.exists("./downloads/"+filename):
    filename, _ = urllib.request.urlretrieve(url + filename, "./downloads/"+filename)
  return filename

def read_data(filename):
  """Extract the first file enclosed in a zip file as a list of words."""
  with zipfile.ZipFile("./downloads/"+filename) as f:
    data = tf.compat.as_str(f.read(f.namelist()[0])).split()
  return data

def build_dataset(words, n_words):
  """Process raw inputs into a dataset."""
  count = [['UNK', -1]]
  count.extend(collections.Counter(words).most_common(n_words - 1))
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
  data = list()
  unk_count = 0
  for word in words:
    if word in dictionary:
      index = dictionary[word]
    else:
      index = 0  # dictionary['UNK']
      unk_count += 1
    data.append(index)
  count[0][1] = unk_count
  reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
  return data, count, dictionary, reversed_dictionary

def generate_batches(data, size, contextWidth, negativeSize):
  cHalfWidth = int(contextWidth/2)
  words = []
  contexts = []
  negatives = []
  index = random.sample(range(cHalfWidth,len(data)-cHalfWidth),size)
  for z in index:
      context = []
      for m in range(-cHalfWidth,cHalfWidth+1):
        if m == 0:
          words.append([data[z]])
        else: 
          context.append(data[z+m])
      contexts.append(context)
      negatives.append(random.sample(data,negativeSize))
  return([np.array(words),np.array(contexts),np.array(negatives)],[np.array([1]*size),np.array([[0]*negativeSize]*size)])

### Downloading Wikipedia text database

In [ ]:
filename = maybe_download('http://mattmahoney.net/dc/', 'text8.zip')
vocabulary = read_data(filename)
print('Number of words: ', len(vocabulary))

### Parameters for skipgram model

In [ ]:
vocabulary_size = 5000
data_index = 0
batch_size = 128
wordvec_dim = 32

skip_window = 3       # How many words to consider left and right.
num_skips = 4         # How many times to reuse an input to generate a label.
context_half = 3
context_size = context_half*2

valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
neg_size = 5    # Number of negative examples to sample.


### Creating a dictionary and reverse dictionary for word embedding

In [ ]:
data, count, dictionary, reverse_dictionary = build_dataset(vocabulary,vocabulary_size)
del vocabulary  # Hint to reduce memory.

### Saving a .tsv label file for viewing in TensorBoard

In [ ]:
with open('./logs/word2vec_label.tsv', 'w') as fr:
    for i in range(vocabulary_size):
        fr.write(reverse_dictionary[i]+'\n')

### Generate training/validation batches

In [ ]:
X,Y = generate_batches(data, 500000, context_size, neg_size)
vX, vY = generate_batches(data, 5000, context_size, neg_size)

### Definining neural model with Keras
Graph of cbow neural model shown below.


![Graph of cbow model](./images/cbowGraph.png)


In [ ]:
word = Input(shape=(1,), name='inputWord')
context = Input(shape=(context_size,), name='inputContext')
negSamples = Input(shape=(neg_size,), name='inputNegatives')

word2vec = Embedding(input_dim=vocabulary_size,output_dim=wordvec_dim, embeddings_initializer='glorot_normal', name='word2vec')

vec_word = word2vec(word)
vec_context = word2vec(context)
vec_negSamples = word2vec(negSamples)
cbow = Lambda(lambda x: K.mean(x, axis=1), name='cbowAverage')(vec_context)

word_context = merge([vec_word, cbow], mode='dot')
negative_context = merge([vec_negSamples, cbow], mode='dot', concat_axis=-1)

model = Model(input=[word,context,negSamples], output=[word_context,negative_context])

In [ ]:
model.compile(optimizer='rmsprop', loss='mse', metrics=['accuracy'])
model.summary()

### Create logs for saving parameters and run training

In [ ]:
tensorboard = TensorBoard(log_dir='./logs/wordvec', 
  batch_size=500, histogram_freq=1, write_images=True, write_grads=False, write_graph=True, embeddings_freq=1)

model_checkpoint = ModelCheckpoint('./logs/model.hdf5')

model.fit(X,Y,epochs=50,batch_size=500,callbacks=[model_checkpoint,tensorboard], validation_data=(vX,vY))